In [ ]:
import tensorflow as tf
import numpy as np
import tfcoreml
import keras
import coremltools

In [2]:
model_path = '../cat_hipsterizer/models/lmks_1.h5'

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph
    
def printTensors(pb_file):

    # read pb into graph_def
    with tf.gfile.GFile(pb_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # import graph_def
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

    # print operations
    for op in graph.get_operations():
        print(op.name)
    
tf.keras.backend.set_learning_phase(0)
model_keras = tf.keras.models.load_model(model_path)
# inputs:  ['dense_input']
print('inputs: ', [input.op.name for input in model_keras.inputs])

# outputs:  ['dense_4/Sigmoid']
print('outputs: ', [output.op.name for output in model_keras.outputs])
frozen_graph = freeze_session(tf.keras.backend.get_session(), output_names=['dense_3/BiasAdd'])
tf.train.write_graph(frozen_graph, './', 'xor.pb', as_text=False)

In [ ]:
tf.keras.backend.set_learning_phase(0)
keras_model = tf.keras.models.load_model(model_path)
with tf.keras.backend.get_session() as sess:
    tf.saved_model.simple_save(
        sess,
        './test.pb',
        inputs={'input': keras_model.input},
        outputs={'output': keras_model.output})

In [ ]:
import sys

from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat

with tf.Session() as sess:
    model_filename ='./xor.pb'
    with tf.gfile.GFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        g_in = tf.import_graph_def(graph_def)
LOGDIR='./logs'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)

In [ ]:
gf = tf.GraphDef()   
m_file = open('./xor.pb','rb')
gf.ParseFromString(m_file.read())

with open('somefile.txt', 'a') as the_file:
    for n in gf.node:
        the_file.write(n.name+'\n')

file = open('somefile.txt','r')
data = file.readlines()
print ("\noutput name = ")
print (data[len(data)-1])

print ("Input name = ")
file.seek ( 0 )
print (file.readline())

In [ ]:
IMAGE_INPUT_NAME_SHAPE = {'input_1':[1,224,224,3]}
IMAGE_INPUT_SCALE = 1.0/255.0
model = tfcoreml.convert(tf_model_path='./xor.pb',
                         input_name_shape_dict=IMAGE_INPUT_NAME_SHAPE,
                         image_scale=IMAGE_INPUT_SCALE,
                         output_feature_names=['dense_3/BiasAdd'])
model.save('./findCat.mlmodel')

In [ ]:
fake_image = np.random.rand(3, 224, 224)
model.predict({'flatten_input': fake_image})['Identity']
print(coreml_predictions[:10])

In [3]:
your_model = coremltools.converters.keras.convert(model_path, 
                                                  input_names=['image'], 
                                                  output_names=['output'], 
                                                  image_input_names='image')

your_model.save('catLand.mlmodel')








0 : input_1, <keras.engine.input_layer.InputLayer object at 0x145a96810>
1 : Conv1_pad, <keras.layers.convolutional.ZeroPadding2D object at 0x145a96950>
2 : Conv1, <keras.layers.convolutional.Conv2D object at 0x145a96790>
3 : bn_Conv1, <keras.layers.normalization.BatchNormalization object at 0x145a96b90>
4 : Conv1_relu, <keras.layers.advanced_activations.ReLU object at 0x145a96d90>
5 : expanded_conv_depthwise, <keras.layers.convolutional.DepthwiseConv2D object at 0x145a96bd0>
6 : expanded_conv_depthwise_BN, <keras.layers.normalization.BatchNormalization object at 0x145ab41d0>
7 : expanded_conv_depthwise_relu, <keras.layers.advanced_activations.ReLU object at 0x145ab4490>
8 : expanded_conv_project, <keras.layers.convolutional.Conv2D object at 0x145ab4410>
9 : expanded_conv_project_BN, <keras.layers.normalization.BatchNormalization object at 0x145ab4510>
10 : block_1_expand, <keras.layers.convolutional.Conv2D object at 0x145ab4810>
11 : block_1_expand_BN, <keras.layers.normalizati

77 : block_8_depthwise_BN, <keras.layers.normalization.BatchNormalization object at 0x145ad5d90>
78 : block_8_depthwise_relu, <keras.layers.advanced_activations.ReLU object at 0x145ad5d10>
79 : block_8_project, <keras.layers.convolutional.Conv2D object at 0x145ad5f90>
80 : block_8_project_BN, <keras.layers.normalization.BatchNormalization object at 0x145adb110>
81 : block_8_add, <keras.layers.merge.Add object at 0x145adb410>
82 : block_9_expand, <keras.layers.convolutional.Conv2D object at 0x145adb390>
83 : block_9_expand_BN, <keras.layers.normalization.BatchNormalization object at 0x145adb490>
84 : block_9_expand_relu, <keras.layers.advanced_activations.ReLU object at 0x145adb790>
85 : block_9_depthwise, <keras.layers.convolutional.DepthwiseConv2D object at 0x145adb710>
86 : block_9_depthwise_BN, <keras.layers.normalization.BatchNormalization object at 0x145adb910>
87 : block_9_depthwise_relu, <keras.layers.advanced_activations.ReLU object at 0x145adbbd0>
88 : block_9_project, <keras.

In [ ]:
%tensorboard --logdir logs

In [ ]:
%load_ext tensorboard